In [2]:
import os
import pandas as pd
# os.getcwd()
path = 'C:/Users/doudi/OneDrive/Documents/jpconf2020/ppt_movie/'
os.chdir(path)

file = 'PTT_movie_seg.csv'
f = pd.read_csv(file)
f.head()

,tag,label,content,content_seg,content_no_stw
0,P,0,看 完 電影 後 似乎 做 上 時光 機回 歸到 年輕 的 熱血 想要 把 當下 的 感動 ...,"['看', '完', '電影', '後', '似乎', '做', '上', '時光', '機...","['看', '完', '電影', '似乎', '時光', '機回', '歸到', '年輕',..."
1,P,0,防 有雷 這部 電影 算是 今年 來還 不錯 好看 的 而 這个 不斷 重 復 的 概念 也...,"['防', '有雷', '這部', '電影', '算是', '今年', '來還', '不錯'...","['防', '有雷', '這部', '電影', '算是', '來還', '不錯', '好看'..."
2,P,0,詳細 劇情 就 不談 了 直接 說 我 的 想法 吧 我 本身 是 一個 保險 業務員 看到...,"['詳細', '劇情', '就', '不談', '了', '直接', '說', '我', '...","['詳細', '劇情', '不談', '直接', '想法', '吧', '本身', '一個'..."
3,P,0,賽 德克 巴萊 太陽旗 時光 之硯 部落 格 個人 板 劇照 圖文 版 如果 魏德聖 只是 ...,"['賽', '德克', '巴萊', '太陽旗', '時光', '之硯', '部落', '格'...","['賽', '德克', '巴萊', '太陽旗', '時光', '之硯', '部落', '格'..."
4,P,0,以下 爆雷慎入 今天 冒著風 大雨 大去 看 了 高年級 實習生 結果 看 完 大遠 百 早...,"['以下', '爆雷慎入', '今天', '冒著風', '大雨', '大去', '看', '...","['以下', '爆雷慎入', '冒著風', '大雨', '大去', '看', '高年級', ..."


In [3]:
from sklearn.model_selection import train_test_split 
from sklearn import metrics
from keras.preprocessing import sequence
from keras.preprocessing.text import Tokenizer
from keras.utils import np_utils
from keras import optimizers
from keras.layers.merge import concatenate
from keras.models import Sequential, Model
from keras.layers import Input
from keras.layers import Dense, Embedding
from keras.layers import Convolution1D, Flatten, Dropout, MaxPool1D, GlobalAveragePooling1D
from keras import initializers
from keras.layers.core import Dense, Dropout, Activation, Flatten
from keras.layers.embeddings import Embedding
from keras.layers.recurrent import SimpleRNN

Using TensorFlow backend.
C:\Users\doudi\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\doudi\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\doudi\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\doudi\Anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py

In [ ]:
def textCNN(texts, labels):
    training_text, test_text, training_label, test_label = train_test_split(texts, labels, test_size=0.1)

    token = Tokenizer(num_words = 20000)
    token.fit_on_texts(training_text)
    vocab = token.word_index

    x_train_seq = token.texts_to_sequences(training_text)
    x_test_seq = token.texts_to_sequences(test_text)
    x_train = sequence.pad_sequences(x_train_seq, maxlen = 150)
    x_test = sequence.pad_sequences(x_test_seq, maxlen = 150)

    y_train = np_utils.to_categorical(training_label)
    y_test = np_utils.to_categorical(test_label)

    num_labels = 2
    main_input = Input(shape=(150,), dtype='float64')
    # pre-train embeddings
    # embedder = Embedding(len(vocab) + 1, 300, input_length = 20, weights = [embedding_matrix], trainable = False)
    # embed = embedder(main_input)

    embedder = Embedding(len(vocab)+1, 300, input_length=150)
    embed = embedder(main_input)

    # filter size, region size
    cnn = Convolution1D(2, 2, padding='same', strides = 1, activation='relu')(embed)
    cnn = MaxPool1D(pool_size=4)(cnn)
    flat = Flatten()(cnn)
    drop = Dropout(0.2)(flat)
    main_output = Dense(num_labels, activation='sigmoid')(drop)
    model = Model(inputs = main_input, outputs = main_output)
    model.summary()
    
    optmzr = optimizers.Adam(lr=0.001)
    model.compile(loss = 'binary_crossentropy', optimizer=optmzr, metrics = ['accuracy'])
    train_history = model.fit(x_train, y_train, batch_size = 128, epochs = 100, 
                              verbose = 2, validation_data=(x_test, y_test))
    pre_probability = model.predict(x_test)
    predicted = pre_probability.argmax(axis=-1)

    from sklearn import metrics
    print("Classification report for classifier:\n%s\n"
          % ( metrics.classification_report(test_label, predicted)))

    import matplotlib.pyplot as plt
    plt.plot(train_history.history['accuracy'])
    plt.plot(train_history.history['val_accuracy'])
    plt.title('Train History')
    plt.ylabel('accuracy')
    plt.xlabel('Epoch')
    plt.legend(['train','validation'], loc = "upper left")
    plt.show()

    plt.plot(train_history.history['loss'])
    plt.plot(train_history.history['val_loss'])
    plt.title('Train History')
    plt.ylabel('loss')
    plt.xlabel('Epoch')
    plt.legend(['train','validation'], loc = "upper left")
    plt.show()

In [ ]:
def rnn(texts, labels):
    training_text, test_text, training_label, test_label = train_test_split(texts, labels, test_size=0.1)

    token = Tokenizer(num_words = 4000)
    token.fit_on_texts(training_text)
    print(token.document_count)

    x_train_seq = token.texts_to_sequences(training_text)
    x_test_seq = token.texts_to_sequences(test_text)
    x_train = sequence.pad_sequences(x_train_seq, maxlen = 400)
    x_test = sequence.pad_sequences(x_test_seq, maxlen = 400)

    y_train = np_utils.to_categorical(training_label)
    y_test = np_utils.to_categorical(test_label)

    model = Sequential()
    model.add(Embedding(output_dim=32,input_dim=4000,input_length=400))
    model.add(Dropout(0.35))
    model.add(SimpleRNN(units=16))
    model.add(Dense(units=256,activation='relu'))
    model.add(Dropout(0.35))
    model.add(Dense(units=128,activation='relu'))
    model.add(Dense(units=8,activation='sigmoid'))
    model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
    # checkpoint
    # filepath="C:/Users/doudi/OneDrive/Documents/TMU-GIDS/Lab/Competition/AI cup 2019/weights.best.hdf5"
    # checkpoint= ModelCheckpoint(filepath, monitor='val_accuracy', verbose=1, save_best_only=True, mode='max')
    train_history = model.fit(x_train, y_train, validation_data=(x_test, y_test), 
                              epochs=100, batch_size=128, verbose=1, validation_split=0.2)
    scores = model.evaluate(x_test, y_test, verbose=2)
    print(scores[1])
    
    pre_probability = model.predict(x_test)
    predicted = pre_probability.argmax(axis=-1)

    from sklearn import metrics
    print("Classification report for classifier:\n%s\n"
          % ( metrics.classification_report(test_label, predicted)))

    import matplotlib.pyplot as plt
    plt.plot(train_history.history['accuracy'])
    plt.plot(train_history.history['val_accuracy'])
    plt.title('Train History')
    plt.ylabel('accuracy')
    plt.xlabel('Epoch')
    plt.legend(['train','validation'], loc = "upper left")
    plt.show()

    plt.plot(train_history.history['loss'])
    plt.plot(train_history.history['val_loss'])
    plt.title('Train History')
    plt.ylabel('loss')
    plt.xlabel('Epoch')
    plt.legend(['train','validation'], loc = "upper left")
    plt.show()